# Model Training

## Requirements

In [6]:
## Load requirements
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
import plotly
import gc
import pickle
import matplotlib.pyplot as plt 

from typing import Tuple
from datetime import timedelta, datetime

## Utilities

In [7]:
def slicer(df:pd.DataFrame) -> Tuple[pd.DataFrame, ...]:
    """
    Generates timeseries slicing for training
    """
    startDate = df.index.min()
    endDate = df.index.max()

    percTrain = .7 #porcentaje del subset
    percVal = .2
    percTest = .1

    difDate = (endDate - startDate).days

    startTest = endDate - timedelta(days = difDate*percTest)
    startVal = startTest - timedelta(days = difDate*percVal)

    df_train = df[df.index < startVal]#.set_index('bd_business_date')
    df_val = df[(df.index > startVal) & (df.index < startTest) ]#.set_index('bd_business_date')
    df_test = df[df.index > startTest]#.set_index('bd_business_date')

    return (df_train, df_val, df_test)

## Data Load

In [4]:
csv_path = "../stock_forecasting/stock_data/Compiled/all_stocks_2006-01-01_to_2018-01-01.csv"
df = pd.read_csv(csv_path)

## Model Training

In [3]:
## refactor code

## ciclo for por stock name
    #model training
    #model fitting
    #append score
    #export model to .pkl (pickle.dump)
    
    #incorporar try: except: pass

